In [ ]:
# Cell 2: Download Citi Bike Data for 2024
# Purpose: Fetch monthly ZIP files for 2024 containing trip data
import requests
import os

def download_citibike_data(year, month):
    month_str = f"{month:02d}"
    #url = f"https://s3.amazonaws.com/tripdata/JC-{year}{month_str}-citibike-tripdata.csv.zip" 
    url = f"https://s3.amazonaws.com/tripdata/{year}{month_str}-citibike-tripdata.zip" 
    zip_file_path = os.path.join( "raw_data","2024", f"{year}{month_str}-citibike-tripdata.csv.zip")
    
    if os.path.exists(zip_file_path):
        print(f"File for {year}-{month_str} already exists")
        return zip_file_path
    
    print(f"Downloading data for {year}-{month_str}...")
    response = requests.get(url)
    response.raise_for_status()
    with open(zip_file_path, "wb") as f:
        f.write(response.content)
    print(f"Downloaded {zip_file_path}")
    return zip_file_path

# Download data for 2024 (12 months)
year = 2024
zip_paths = [download_citibike_data(year, month) for month in range(1, 13)]
print("All data downloaded successfully")

In [ ]:
# Cell 2: Download Citi Bike Data for 2024
# Purpose: Fetch monthly ZIP files for 2024 containing trip data


In [ ]:
import os
import zipfile
import pandas as pd

# Set paths
folder_path = "raw_data/2024"
output_folder = "filter_data_2024"
os.makedirs(output_folder, exist_ok=True)

# File list
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".zip")])

# Track column structure
column_set = None
matching_files = []
null_counts = {}
row_counts = {}

# Process ZIPs
for file_name in file_list:
    month = file_name.split("-")[0]
    file_path = os.path.join(folder_path, file_name)
    monthly_df_list = []

    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        csv_files = [f for f in zip_ref.namelist() if f.endswith(".csv") and "__MACOSX" not in f]
        if not csv_files:
            print(f"⚠️ No CSVs in {file_name}")
            continue

        for csv_name in csv_files:
            with zip_ref.open(csv_name) as csv_file:
                try:
                    df = pd.read_csv(csv_file, low_memory=False)

                    # Drop index column if present
                    if "Unnamed: 0" in df.columns:
                        df.drop(columns=["Unnamed: 0"], inplace=True)

                    # Drop any column with "duplicate" in the name
                    dup_cols = [col for col in df.columns if "duplicate" in col.lower()]
                    if dup_cols:
                        print(f"⚠️ Dropping duplicate columns in {csv_name}: {dup_cols}")
                        df.drop(columns=dup_cols, inplace=True)

                    # Check column consistency
                    if column_set is None:
                        column_set = set(df.columns)
                        matching_files.append(file_name)
                    elif set(df.columns) == column_set:
                        matching_files.append(file_name)
                    else:
                        print(f"❌ {file_name} → {csv_name} has different columns, skipping.")
                        continue

                    # Save raw stats
                    row_counts[csv_name] = len(df)
                    null_counts[csv_name] = df.isnull().sum().to_dict()

                    monthly_df_list.append(df)

                except Exception as e:
                    print(f"❌ Failed to process {csv_name} in {file_name}: {e}")

    # Combine and save cleaned data
    if monthly_df_list:
        combined_df = pd.concat(monthly_df_list, ignore_index=True)
        cleaned_df = combined_df.dropna()

        output_csv_name = f"{month}_filtered.csv"
        output_path = os.path.join(output_folder, output_csv_name)
        cleaned_df.to_csv(output_path, index=False)

        print(f"✅ Saved: {output_csv_name} with {len(cleaned_df)} cleaned rows")

# Summary
print("\n--- Summary ---")
print("\nFiles with matching columns:")
print(matching_files)

print("\nRow counts per original CSV file:")
for file, count in row_counts.items():
    print(f"{file}: {count} rows")

print("\nNull values per original CSV file:")
for file, counts in null_counts.items():
    print(f"\n{file}:")
    for col, val in counts.items():
        print(f"  {col}: {val}")


In [ ]:
import os
import pandas as pd

# Set the folder where your .csv files are located
folder_path = "filter_data_2024"
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".csv")])

column_set = None
matching_files = []
null_counts = {}
row_counts = {}

for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    try:
        df = pd.read_csv(file_path)

        # Initialize or compare column set
        if column_set is None:
            column_set = set(df.columns)
            matching_files.append(file_name)
        elif set(df.columns) == column_set:
            matching_files.append(file_name)
        else:
            print(f"❌ {file_name} has different columns")

        # Count null values per column
        null_counts[file_name] = df.isnull().sum().to_dict()

        # Count number of rows
        row_counts[file_name] = len(df)

    except Exception as e:
        print(f"⚠️ Error reading {file_name}: {e}")

# Display summary
print("\n✅ Files with matching columns:")
print(matching_files)

print("\n📊 Row counts per file:")
for file, count in row_counts.items():
    print(f"{file}: {count} rows")

print("\n🧼 Null values per file:")
for file, counts in null_counts.items():
    print(f"\n{file}:")
    for col, val in counts.items():
        print(f"  {col}: {val}")


In [ ]:
import os
import pandas as pd

def combine_monthly_csvs(folder_path: str, output_file: str) -> None:
    """
    Combines all CSV files from the folder into a single CSV file.
    
    Args:
        folder_path (str): Directory containing monthly CSV files.
        output_file (str): Output path for the combined CSV file.
    """
    all_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]
    combined_df = []

    for file in sorted(all_files):
        file_path = os.path.join(folder_path, file)
        print(f"Loading: {file}")
        df = pd.read_csv(file_path, low_memory=False)
        combined_df.append(df)

    # Concatenate all and write to a single CSV
    result = pd.concat(combined_df, ignore_index=True)
    result.to_csv(output_file, index=False)
    print(f"✅ Combined CSV saved as: {output_file}")

# 👇 Run the combiner
combine_monthly_csvs("filter_data_2024", "all_months_combined.csv")



In [ ]:
import pandas as pd

# Load the combined dataset
file_path = r"D:\Code\Git\ESDS_500_Final\all_months_combined.csv"
df = pd.read_csv(file_path, usecols=["start_station_id"])

# Drop missing station IDs and ensure string type
df = df.dropna(subset=["start_station_id"])
df["start_station_id"] = df["start_station_id"].astype(str)

# Find top 3 most frequent start stations
top_3_stations = (
    df["start_station_id"]
    .value_counts()
    .head(3)
    .rename_axis("start_station_id")
    .reset_index(name="trip_count")
)


print("Top 3 Start Stations by Trip Count:")
print(top_3_stations)


In [16]:
import pandas as pd

# === 1. Load the combined dataset ===
file_path = r"D:\Code\Git\ESDS_500_Final\all_months_combined.csv"
df = pd.read_csv(file_path)

# === 2. Drop missing start station IDs and convert to string ===
df = df.dropna(subset=["start_station_id"])
df["start_station_id"] = df["start_station_id"].astype(str)

# === 3. Find top 3 most frequent start stations ===
top_3_stations = (
    df["start_station_id"]
    .value_counts()
    .head(3)
    .rename_axis("start_station_id")
    .reset_index(name="trip_count")
)

# === 4. Extract top station IDs into a list ===
top_station_ids = top_3_stations["start_station_id"].tolist()

# === 5. Filter original DataFrame for only top 3 start stations ===
df_top_stations = df[df["start_station_id"].isin(top_station_ids)]

# === 6. Save the filtered data to a new CSV file ===
output_path = r"D:\Code\Git\ESDS_500_Final\BikeRide2024Top3Location.csv"
df_top_stations.to_csv(output_path, index=False)

print(f"✅ Filtered data for top 3 stations saved to: {output_path}")


KeyboardInterrupt: 